In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import expr, date_format, when, col, to_timestamp, array, count, first, month, to_date, dayofweek,\
datediff, current_date, date_add, max, year, percent_rank
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer,OneHotEncoder, IndexToString, VectorAssembler
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import FloatType
from pyspark.ml.evaluation import RegressionEvaluator

In [1]:
#

In [7]:
spark = SparkSession.builder.appName('GCPUnzipExample').getOrCreate()
df_taxi = spark.read.option('header', 'true').parquet('gs://msca-bdp-student-gcs/unzipped-taxi-data/nyc-taxi-data/',inferSchema=True)
df_csv = spark.read.csv("gs://msca-bdp-student-gcs/unzipped-taxi-data/taxi+_zone_lookup.csv", header=True)

In [8]:
df_taxi.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [3]:
df_taxi.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2011-01-01 00:10:00|  2011-01-01 00:12:00|              4|          0.0|         1|              null|         145|         145|           1|        2.9|  0.5|    0.5|      0.2

In [ ]:
# df = df_taxi.select("tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","PULocationID","DOLocationID",\
                    # "total_amount")

In [9]:
# dayofweek, 1:Sunday, 2:Monday, 7:Saturday
# weekdays, 1:weekdays, 0:weekends
# duration, in use of cleaning data
df_taxi_add = (df_taxi.withColumn("dayofweek", dayofweek("tpep_pickup_datetime"))\
               .withColumn("pickup_time", date_format(col("tpep_pickup_datetime"), "HH:mm:ss"))\
               .withColumn("dropoff_time", date_format(col("tpep_dropoff_datetime"), "HH:mm:ss"))\
               .withColumn("month", month(col("tpep_pickup_datetime")))\
               .withColumn("year", year("tpep_pickup_datetime"))\
               .withColumn("weekdays", when(col("dayofweek").isin(1,7), "0").otherwise("1"))\
                .withColumn("durations", F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")))

In [10]:
df_taxi_add.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+-----------+------------+-----+----+--------+---------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|dayofweek|pickup_time|dropoff_time|month|year|weekdays|durations|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+-----------+------------+-----+----+--------+---

In [15]:
df_taxi_add.filter(F.col("year") == 2015).show(2)

23/11/14 22:15:55 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 79.0 in stage 14.0 (TID 397) (hub-msca-bdp-dphub-students-haoyu1-w-1.c.msca-bdp-student-ap.internal executor 4): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainDoubleDictionary
	at org.apache.parquet.column.Dictionary.decodeToLong(Dictionary.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToLong(ParquetDictionary.java:36)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getLong(OnHeapColumnVector.java:364)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(

Py4JJavaError: An error occurred while calling o200.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 79 in stage 14.0 failed 10 times, most recent failure: Lost task 79.9 in stage 14.0 (TID 426) (hub-msca-bdp-dphub-students-haoyu1-w-1.c.msca-bdp-student-ap.internal executor 4): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainDoubleDictionary
	at org.apache.parquet.column.Dictionary.decodeToLong(Dictionary.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToLong(ParquetDictionary.java:36)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getLong(OnHeapColumnVector.java:364)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodege

In [11]:
df_taxi_add.count()

1368245364

In [12]:
# cleaning unreasonable data
df_taxi_clean = df_taxi_add.filter(((F.col("durations") > 0) & (F.col("durations") < (12 * 3600)))\
                                   & (F.col("total_amount") > 0)\
                                   & ((F.col("passenger_count") > 0) & (F.col("passenger_count") < 7))\
                                   & (F.col("PULocationID").isin(1,263))\
                                   & (F.col("DOLocationID").isin(1,263)))

In [13]:
df_taxi_clean.count()

23/11/14 22:05:27 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 84.0 in stage 10.0 (TID 245) (hub-msca-bdp-dphub-students-haoyu1-w-1.c.msca-bdp-student-ap.internal executor 1): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainDoubleDictionary
	at org.apache.parquet.column.Dictionary.decodeToLong(Dictionary.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToLong(ParquetDictionary.java:36)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getLong(OnHeapColumnVector.java:364)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.Who

Py4JJavaError: An error occurred while calling o195.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 84 in stage 10.0 failed 10 times, most recent failure: Lost task 84.9 in stage 10.0 (TID 287) (hub-msca-bdp-dphub-students-haoyu1-w-1.c.msca-bdp-student-ap.internal executor 1): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainDoubleDictionary
	at org.apache.parquet.column.Dictionary.decodeToLong(Dictionary.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToLong(ParquetDictionary.java:36)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getLong(OnHeapColumnVector.java:364)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.

In [18]:
# add time in seconds and time bins(15 mins per bin)
def time_to_seconds(time_col):
    return (
        expr("HOUR(" + time_col + ")") * 3600 +
        expr("MINUTE(" + time_col + ")") * 60 +
        expr("SECOND(" + time_col + ")")
    )
bin_size = 15 * 60 # 15mins
df_with_seconds = df_taxi_clean.withColumn("pickup_time_in_seconds", time_to_seconds("pickup_time"))
df_with_time_bins = df_with_seconds.withColumn("pickup_time_bins", (col("pickup_time_in_seconds") / bin_size).cast("integer"))

In [19]:
df_with_time_bins.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+-----------+------------+-----+----+--------+---------+----------------------+----------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|dayofweek|pickup_time|dropoff_time|month|year|weekdays|durations|pickup_time_in_seconds|pickup_time_bins|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+-----------

In [21]:
# add y variable
df_with_count = df_with_time_bins.groupby("year","month","dayofweek","PULocationID","pickup_time_bins")\
                    .agg(count("*").alias("number_of_pickups"))
df_with_count.show()

+----+-----+---------+------------+----------------+-----------------+
|year|month|dayofweek|PULocationID|pickup_time_bins|number_of_pickups|
+----+-----+---------+------------+----------------+-----------------+
|2011|    1|        7|         106|               3|               16|
|2011|    1|        7|         171|               5|                3|
|2011|    1|        7|         164|               8|              646|
|2011|    1|        7|         237|              14|              100|
|2011|    1|        7|         262|              14|               66|
|2011|    1|        7|         100|              13|              159|
|2011|    1|        7|         141|              18|              181|
|2011|    1|        7|         168|              21|                7|
|2011|    1|        7|         142|              21|               87|
|2011|    1|        7|         262|              22|               43|
|2011|    1|        7|          40|              25|                7|
|2011|

In [26]:
# add y variable to original dataset
joined_df_taxi = df_with_time_bins.join(df_with_count, ["year","month","dayofweek","PULocationID","pickup_time_bins"],'left')
joined_df_taxi.show()

+----+-----+---------+------------+----------------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+--------+---------+----------------------+-----------------+
|year|month|dayofweek|PULocationID|pickup_time_bins|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|pickup_time|dropoff_time|weekdays|durations|pickup_time_in_seconds|number_of_pickups|
+----+-----+---------+------------+----------------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+-----------+-----+----

In [42]:
# select columns that are useful to the model
df_nyc_taxi = joined_df_taxi.select("year", "month", "dayofweek", "weekdays", "PULocationID", "pickup_time_bins",\
                                    "number_of_pickups")

In [43]:
df_nyc_taxi.show()

+----+-----+---------+--------+------------+----------------+-----------------+
|year|month|dayofweek|weekdays|PULocationID|pickup_time_bins|number_of_pickups|
+----+-----+---------+--------+------------+----------------+-----------------+
|2011|    1|        7|       0|         145|               0|               50|
|2011|    1|        7|       0|         264|               0|              684|
|2011|    1|        7|       0|         264|               0|              684|
|2011|    1|        7|       0|         146|               0|               49|
|2011|    1|        7|       0|         138|               3|               93|
|2011|    1|        7|       0|         170|               1|             1056|
|2011|    1|        7|       0|         237|               2|              495|
|2011|    1|        7|       0|         170|               3|              930|
|2011|    1|        7|       0|          90|               2|              729|
|2011|    1|        7|       0|         

In [44]:
df_nyc_taxi.select("pickup_time_bins").distinct().count()

96

In [45]:
df_nyc_taxi = df_nyc_taxi.withColumn("PULocationID",df_nyc_taxi.PULocationID.cast(StringType()))

In [46]:
df_nyc_taxi.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- weekdays: string (nullable = false)
 |-- PULocationID: string (nullable = true)
 |-- pickup_time_bins: integer (nullable = true)
 |-- number_of_pickups: long (nullable = true)



In [47]:
df_nyc_taxi = df_nyc_taxi.dropna()

In [ ]:
#convert relevant categorical into one hot encoded
indexer1 = StringIndexer(inputCol="PULocationID", outputCol="LocationIdx").setHandleInvalid("skip")
indexer2 = StringIndexer(inputCol="weekdays", outputCol="WeekdaysIdx").setHandleInvalid("skip")
indexer3 = StringIndexer(inputCol="dayofweek", outputCol="DayofweekIdx").setHandleInvalid("skip")
#gather all indexers as inputs to the One Hot Encoder
inputs = [indexer1.getOutputCol(),indexer2.getOutputCol(), \
          indexer3.getOutputCol()]

#create the one hot encoder
encoder = OneHotEncoder(inputCols=inputs,  \
                                 outputCols=["LocationVec","WeekdaysVec","DayofweekVec"])

#run it through a pipeline
pipeline = Pipeline(stages=[indexer1,indexer2,indexer3,encoder])
df_nyc_taxi = pipeline.fit(df_nyc_taxi).transform(df_nyc_taxi)
df_nyc_taxi.show()

In [ ]:
vectorAssembler = VectorAssembler(inputCols=[ "year","month","pickup_time_bins",
                                              "LocationVec", "WeekdaysVec", "DayofweekVec"],
                                  outputCol="features")
df_nyc_taxi = vectorAssembler.transform(df_nyc_taxi)

In [ ]:
# Split the data into training and test sets.
train_df_taxi, test_df_taxi =  df_nyc_taxi.randomSplit([0.7, 0.3],0.0)